# Using Experimental Simulators with Q# and Python

In [ ]:
from azure.quantum import Workspace

workspace = Workspace (
    subscription_id = "test",
    resource_group = "test",
    name = "test",
    location = "test"
)

In [1]:
import qutip as qt
import qsharp.experimental
qsharp.experimental.enable_noisy_simulation()

qsharp.config["experimental.simulators.nQubits"] = 1
qsharp.config["experimental.simulators.stabilizerStateStyle"] = "denseGroupPresentation"

Preparing Q# environment...
....

In [2]:
%%qsharp
namespace Script {    open Microsoft.Quantum.Diagnostics;
    
    operation DumpPlus() : Unit {
        use q = Qubit();
        H(q);
        DumpMachine();
        X(q);
        Reset(q);
    }
}

## Depolarizing Noise

In [3]:
noise_model = qsharp.experimental.get_noise_model_by_name('ideal')

In [4]:
def depolarizing_noise(p=1.0):
    return p * qt.to_super(qt.qeye(2)) + ((1 - p) / 4) * sum(map(
        qt.to_super, [P.as_qobj() for P in qsharp.Pauli]
    ))

In [5]:
noise_model['h'] = depolarizing_noise(0.99) * qt.to_super(qt.qip.operations.hadamard_transform())
qsharp.experimental.set_noise_model(noise_model)

In [6]:
DumpPlus.simulate()

|0⟩	0.7071067811865476 + 0𝑖
|1⟩	0.7071067811865476 + 0𝑖

()

In [18]:
DumpPlus.simulate_noise()

# of qubits,2
State data,"$$\left\langle X𝟙, 𝟙Z \right\rangle$$"


()

## Stabilizer Simulation

In [17]:
noise_model = qsharp.experimental.get_noise_model_by_name('ideal_stabilizer')
noise_model['h'] = qsharp.experimental.SequenceProcess(1, 
    [
        qsharp.experimental.ChpDecompositionProcess(1, [
            qsharp.experimental.Hadamard(0)
        ]),
        qsharp.experimental.MixedPauliProcess(1, [
            (0.8, 'I'),
            (0.2, 'Z')
        ])
    ]
)
qsharp.experimental.set_noise_model(noise_model)
qsharp.config['experimental.simulators.representation'] = 'stabilizer'
qsharp.config['experimental.simulators.nQubits'] = 2

In [16]:
%%qsharp
namespace Script {    operation DumpBellPair() : Unit {
        use left = Qubit();
        use right = Qubit();
        within {
            H(left);
            CNOT(left, right);
        } apply {
            Microsoft.Quantum.Diagnostics.DumpMachine();
        }
    }
}

In [15]:
DumpBellPair.simulate()

|0⟩	0.7071067811865476 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0 + 0𝑖
|3⟩	0.7071067811865476 + 0𝑖

()

In [14]:
for _ in range(5):
    DumpBellPair.simulate_noise()

# of qubits,2
State data,"$$\left\langle XX, ZZ \right\rangle$$"


# of qubits,2
State data,"$$\left\langle XX, ZZ \right\rangle$$"


# of qubits,2
State data,"$$\left\langle XX, ZZ \right\rangle$$"


# of qubits,2
State data,"$$\left\langle XX, ZZ \right\rangle$$"


# of qubits,2
State data,"$$\left\langle XX, ZZ \right\rangle$$"
